In [ ]:
from rasterfoundry.api import API
refresh_token = '<Refresh Token>'
api = API(refresh_token=refresh_token)

In [ ]:
analyses = api.analyses
analysis = analyses[0]
analyses

In [ ]:
m = analysis.get_map()

projects = api.projects
project = projects[0]
project.compare(analysis, m)
m